In [1]:
# Preprocessing Data for suggroate model 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import seaborn as sns
import sklearn
import imblearn
import matplotlib.pyplot as plt
import time
import sklearn.metrics as m
import xgboost as xgb
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# read the data
data_target = pd.read_csv('../datasets/target_model/CICIDS2017/df_target_0311.csv')
data_sugrrogate = pd.read_csv('../datasets/surrogate_model/CICIDS2017/df_surrogate_0311.csv') # datasets\surrogate_model\CICIDS2017\df_surrogate_0311.csv
df = pd.concat([data_target, data_sugrrogate], axis=0)
df



,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Binary_class,attack_type
0,80,5435785,4,0,24,0,6,6,6.000000,0.000000,...,0.0,952,952,5434833.0,0.0,5434833,5434833,DDoS,malicious,DOS
1,6668,26,1,1,2,6,2,2,2.000000,0.000000,...,0.0,0,0,0.0,0.0,0,0,PortScan,malicious,PortScan
2,53,23475,1,1,70,152,70,70,70.000000,0.000000,...,0.0,0,0,0.0,0.0,0,0,BENIGN,benign,benign
3,80,37553,2,0,0,0,0,0,0.000000,0.000000,...,0.0,0,0,0.0,0.0,0,0,BENIGN,benign,benign
4,53,71220,4,4,172,384,43,43,43.000000,0.000000,...,0.0,0,0,0.0,0.0,0,0,BENIGN,benign,benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261180,53,23712,2,2,82,190,41,41,41.000000,0.000000,...,0.0,0,0,0.0,0.0,0,0,BENIGN,benign,benign
1261181,80,97890396,6,6,357,11595,357,0,59.500000,145.744640,...,0.0,292,292,97900000.0,0.0,97900000,97900000,DoS Hulk,NaN,DoS
1261182,60040,32,1,1,0,0,0,0,0.000000,0.000000,...,0.0,0,0,0.0,0.0,0,0,BENIGN,benign,benign
1261183,443,11408070,19,21,10829,22666,3638,0,569.947368,1303.224781,...,0.0,1408037,1408037,9999813.0,0.0,9999813,9999813,BENIGN,benign,benign


In [3]:
# Preprocessing Data
# Removing whitespaces in column names.
col_names = [col.replace(' ', '') for col in df.columns]
df.columns = col_names


# Replace weird characters
label_names = df['Label'].unique()

import re
label_names = [re.sub("[^a-zA-Z ]+", "", l) for l in label_names]
label_names = [re.sub("[\s\s]", '_', l) for l in label_names]
label_names = [lab.replace("__", "_") for lab in label_names]
label_names, len(label_names)

# Replacing 'Label' column values with new readable values.
labels = df['Label'].unique()
for i in range(0,len(label_names)):
    df['Label'] = df['Label'].replace({labels[i] : label_names[i]})
    
df['Label'].unique()

df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

In [4]:
df.value_counts('Label')

Label
BENIGN                      2095057
DDoS                         128014
PortScan                      90694
DoS_GoldenEye                 10286
FTPPatator                     5931
DoS_slowloris                  5385
DoS_Slowhttptest               5228
SSHPatator                     3219
Bot                            1948
Web_Attack_Brute_Force         1470
Web_Attack_XSS                  652
Infiltration                     36
Web_Attack_Sql_Injection         21
Heartbleed                       11
Name: count, dtype: int64

In [5]:
df_benign = df[df['Label'] == 'BENIGN']
df_benign = df_benign.sample(frac=0.05).reset_index(drop=True)
df_DOS_GoldenEye = df[df['Label'] == 'DoS_GoldenEye']
df_DOS_slowloris = df[df['Label'] == 'DoS_slowloris']
df_DOS_slowhttptest = df[df['Label'] == 'DoS_slowhttptest']
df_DOS_GoldenEye

,DestinationPort,FlowDuration,TotalFwdPackets,TotalBackwardPackets,TotalLengthofFwdPackets,TotalLengthofBwdPackets,FwdPacketLengthMax,FwdPacketLengthMin,FwdPacketLengthMean,FwdPacketLengthStd,...,ActiveStd,ActiveMax,ActiveMin,IdleMean,IdleStd,IdleMax,IdleMin,Label,Binary_class,attack_type
400,80,35190473,2,0,0,0,0,0,0.000000,0.000000,...,0.0,0,0,35200000.0,0.0,35200000,35200000,DoS_GoldenEye,malicious,DoS
534,80,10081946,7,6,521,11632,521,0,74.428571,196.919490,...,0.0,821,821,5077896.0,0.0,5077896,5077896,DoS_GoldenEye,malicious,DoS
856,80,10623256,9,6,349,11632,349,0,38.777778,116.333333,...,0.0,650,650,5617496.0,0.0,5617496,5617496,DoS_GoldenEye,malicious,DoS
1046,80,5006466,9,7,590,11632,590,0,65.555556,196.666667,...,0.0,0,0,0.0,0.0,0,0,DoS_GoldenEye,malicious,DoS
1170,80,11864685,5,5,366,3525,366,0,73.200000,163.680176,...,0.0,847,847,6858712.0,0.0,6858712,6858712,DoS_GoldenEye,malicious,DoS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259916,80,11609749,8,6,387,11632,387,0,48.375000,136.825162,...,0.0,884,884,6607161.0,0.0,6607161,6607161,DoS_GoldenEye,malicious,DoS
1260094,80,11275669,14,8,1797,11632,599,0,128.357143,255.063373,...,0.0,415283,415283,5855229.0,0.0,5855229,5855229,DoS_GoldenEye,malicious,DoS
1260181,80,113928908,2,0,0,0,0,0,0.000000,0.000000,...,0.0,0,0,114000000.0,0.0,114000000,114000000,DoS_GoldenEye,malicious,DoS
1260437,80,11686484,5,4,470,3525,470,0,94.000000,210.190390,...,0.0,959,959,6681404.0,0.0,6681404,6681404,DoS_GoldenEye,malicious,DoS


In [6]:
df = pd.concat([df_benign, df_DOS_GoldenEye, df_DOS_slowloris, df_DOS_slowhttptest], axis=0)
df = df.rename(columns={'Binary_class': 'binary_classification'})
df

,DestinationPort,FlowDuration,TotalFwdPackets,TotalBackwardPackets,TotalLengthofFwdPackets,TotalLengthofBwdPackets,FwdPacketLengthMax,FwdPacketLengthMin,FwdPacketLengthMean,FwdPacketLengthStd,...,ActiveStd,ActiveMax,ActiveMin,IdleMean,IdleStd,IdleMax,IdleMin,Label,binary_classification,attack_type
0,443,13503879,12,12,2984,5313,1325,0,248.666667,499.361107,...,0.000,1828437,1828437,10000000.0,0.0,10000000,10000000,BENIGN,benign,benign
1,47763,56,1,1,0,0,0,0,0.000000,0.000000,...,0.000,0,0,0.0,0.0,0,0,BENIGN,benign,benign
2,53,30965,2,2,68,182,34,34,34.000000,0.000000,...,0.000,0,0,0.0,0.0,0,0,BENIGN,benign,benign
3,53,24039,2,2,50,82,25,25,25.000000,0.000000,...,0.000,0,0,0.0,0.0,0,0,BENIGN,benign,benign
4,59569,97572388,5,9,12982,56,11595,0,2596.400000,5065.280279,...,0.000,34389,34389,47900000.0,53800000.0,85900000,9832740,BENIGN,benign,benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255279,80,102645877,14,3,2541,6,231,0,181.500000,98.363337,...,4529979.892,6406845,486,19200000.0,18800000.0,51300000,5966340,DoS_slowloris,malicious,DoS
1258044,80,3003648,3,0,0,0,0,0,0.000000,0.000000,...,0.000,0,0,0.0,0.0,0,0,DoS_slowloris,malicious,DoS
1258419,80,3003925,3,0,0,0,0,0,0.000000,0.000000,...,0.000,0,0,0.0,0.0,0,0,DoS_slowloris,malicious,DoS
1260583,80,102690607,14,3,2541,6,231,0,181.500000,98.363337,...,4532998.530,6411372,744,19300000.0,18800000.0,51300000,5743827,DoS_slowloris,malicious,DoS


In [7]:
df.columns

Index(['DestinationPort', 'FlowDuration', 'TotalFwdPackets',
       'TotalBackwardPackets', 'TotalLengthofFwdPackets',
       'TotalLengthofBwdPackets', 'FwdPacketLengthMax', 'FwdPacketLengthMin',
       'FwdPacketLengthMean', 'FwdPacketLengthStd', 'BwdPacketLengthMax',
       'BwdPacketLengthMin', 'BwdPacketLengthMean', 'BwdPacketLengthStd',
       'FlowBytes/s', 'FlowPackets/s', 'FlowIATMean', 'FlowIATStd',
       'FlowIATMax', 'FlowIATMin', 'FwdIATTotal', 'FwdIATMean', 'FwdIATStd',
       'FwdIATMax', 'FwdIATMin', 'BwdIATTotal', 'BwdIATMean', 'BwdIATStd',
       'BwdIATMax', 'BwdIATMin', 'FwdPSHFlags', 'BwdPSHFlags', 'FwdURGFlags',
       'BwdURGFlags', 'FwdHeaderLength', 'BwdHeaderLength', 'FwdPackets/s',
       'BwdPackets/s', 'MinPacketLength', 'MaxPacketLength',
       'PacketLengthMean', 'PacketLengthStd', 'PacketLengthVariance',
       'FINFlagCount', 'SYNFlagCount', 'RSTFlagCount', 'PSHFlagCount',
       'ACKFlagCount', 'URGFlagCount', 'CWEFlagCount', 'ECEFlagCount',
       '

In [8]:
# drop columns
df_feature = df[['FlowDuration', 'TotalFwdPackets', 'TotalBackwardPackets', 'TotalLengthofFwdPackets', 'TotalBackwardPackets', 
                 'FwdPacketLengthMin', 'FwdPacketLengthStd', 'BwdPacketLengthMax', 'BwdPacketLengthMean'
                 ,'BwdPacketLengthStd', 'FlowBytes/s', 'FlowPackets/s', 'FlowIATStd', 'FwdIATTotal', 'FwdIATMean', 'FwdIATMax', 'BwdIATMean', 'BwdIATStd'
                 , 'BwdIATMax', 'BwdIATMin', 'BwdPSHFlags', 'FwdHeaderLength', 'BwdHeaderLength', 'FwdPackets/s', 'BwdPackets/s', 'MaxPacketLength', 'PacketLengthStd'
                 , 'FINFlagCount', 'SYNFlagCount', 'ACKFlagCount', 'Down/UpRatio', 'AvgBwdSegmentSize', 'FwdHeaderLength.1', 'SubflowFwdPackets', 
                 'SubflowFwdBytes', 'IdleStd', 
                 'SubflowBwdPackets', 'SubflowBwdBytes', 'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'ActiveStd', 'ActiveMax', 'Label', 'binary_classification']]

In [9]:
df_feature.columns = ['FlowDuration', 'TotFwdPkts', 'TotBwdPkts', 'TotLenFwdPkts', 'TotLenBwdPkts', 'FwdPktLenMin', 'FwdPktLenStd', 'BwdPktLenMax', 'BwdPktLenMean'
                 ,'BwdPktLenStd', 'FlowByts/s', 'FlowPkts/s', 'FlowIATStd', 'FwdIATTot', 'FwdIATMean', 'FwdIATMax', 'BwdIATMean', 'BwdIATStd'
                 , 'BwdIATMax', 'BwdIATMin', 'BwdPSHFlags', 'FwdHeaderLen', 'BwdHeaderLen', 'FwdPkts/s', 'BwdPkts/s', 'PktLenMax', 'PktLenStd'
                 , 'FINFlagCnt', 'SYNFlagCnt', 'ACKFlagCnt', 'Down/UpRatio', 'BwdSegSizeAvg', 'FwdHeaderLen', 'SubflowFwdPkts', 'SubflowFwdByts', 'IdleStd', 
                 'SubflowBwdPkts', 'SubflowBwdByts', 'InitBwdWinByts', 'FwdActDataPkts', 'ActiveStd', 'ActiveMax', 'Label', 'binary_classification']

In [10]:
df_feature

,FlowDuration,TotFwdPkts,TotBwdPkts,TotLenFwdPkts,TotLenBwdPkts,FwdPktLenMin,FwdPktLenStd,BwdPktLenMax,BwdPktLenMean,BwdPktLenStd,...,SubflowFwdByts,IdleStd,SubflowBwdPkts,SubflowBwdByts,InitBwdWinByts,FwdActDataPkts,ActiveStd,ActiveMax,Label,binary_classification
0,13503879,12,12,2984,12,0,499.361107,1616,442.750000,650.408281,...,2984,0.0,12,5313,19875,11,0.000,1828437,BENIGN,benign
1,56,1,1,0,1,0,0.000000,0,0.000000,0.000000,...,0,0.0,1,0,310,0,0.000,0,BENIGN,benign
2,30965,2,2,68,2,34,0.000000,91,91.000000,0.000000,...,68,0.0,2,182,-1,1,0.000,0,BENIGN,benign
3,24039,2,2,50,2,25,0.000000,41,41.000000,0.000000,...,50,0.0,2,82,-1,1,0.000,0,BENIGN,benign
4,97572388,5,9,12982,9,0,5065.280279,20,6.222222,5.783117,...,12982,53800000.0,9,56,256,3,0.000,34389,BENIGN,benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255279,102645877,14,3,2541,3,0,98.363337,6,2.000000,3.464102,...,2541,18800000.0,3,6,0,11,4529979.892,6406845,DoS_slowloris,malicious
1258044,3003648,3,0,0,0,0,0.000000,0,0.000000,0.000000,...,0,0.0,0,0,-1,0,0.000,0,DoS_slowloris,malicious
1258419,3003925,3,0,0,0,0,0.000000,0,0.000000,0.000000,...,0,0.0,0,0,-1,0,0.000,0,DoS_slowloris,malicious
1260583,102690607,14,3,2541,3,0,98.363337,6,2.000000,3.464102,...,2541,18800000.0,3,6,0,11,4532998.530,6411372,DoS_slowloris,malicious


In [11]:
# save the data
df_feature.to_csv('../datasets/target_model/CICIDS2017/target_model_CICIDS2017data.csv', index=False) #datasets\target_model\CICIDS2018